In [1]:
# !pip3 install pybullet
# !pip3 install roboticstoolbox-python[collision]
# !pip install roboticstoolbox-python==1.1.0

In [1]:
import swift
import spatialgeometry as sg
import roboticstoolbox as rtb
import spatialmath as sm
import numpy as np
import qpsolvers as qp
import cProfile
from spatialmath.pose3d import SE3

In [2]:
target = sg.Sphere(radius =0.01, pose=sm.SE3(0.3, -0.2, 0.0))
target.T[:3, 3] 

array([ 0.3, -0.2,  0. ])

In [3]:
# Launch the simulator Swift
env = swift.Swift()
env.launch()

# Create a Panda robot object
panda = rtb.models.Panda()

# Set joint angles to ready configuration
panda.q = panda.qr

# Number of joint in the panda which we are controlling
n = 7

# Collisions
## s1: Base plate
s1 = sg.Cuboid(
    scale=[0.60, 1.2, 0.02],
    pose=sm.SE3(0.8, 0, 0.0))

# s2, s3 boundary plate front 1 and 2 robot
s2 = sg.Cuboid(
    scale=[0.60, 0.02, 0.3],
    pose=sm.SE3(0.8, 0.6, 0.15))

s3 = sg.Cuboid(
    scale=[0.60, 0.02, 0.3],
    pose=sm.SE3(0.8, -0.6, 0.15))

# s4, s5 boundary plate left and right robot
s4 = sg.Cuboid(
    scale=[0.02, 1.2, 0.3],
    pose=sm.SE3(0.5, 0, 0.15))

s5 = sg.Cuboid(
    scale=[0.02, 1.2, 0.3],
    pose=sm.SE3(1.1, 0, 0.15))

## s6: Center plate
s6 = sg.Cuboid(
    scale=[0.60, 0.02, 0.3],
    pose=sm.SE3(0.8, 0, 0.15))

## s7: Sphere
#s7 = sg.Sphere(radius =0.05,
#    pose=sm.SE3(0.62, 0, 0.25))

collisions = [s1, s2, s3, s4, s5, s6]


# Make a target
target_1 = sg.Sphere(radius =0.01, pose=sm.SE3(0.7, -0.3, 0.30))
target_2 = sg.Sphere(radius =0.01, pose=sm.SE3(0.7, 0.3, 0.30))
# Add the Panda and shapes to the simulator
env.add(panda)
env.add(s1)
env.add(s2)
env.add(s3)
env.add(s4)
env.add(s5)
env.add(s6)
#env.add(s7)
env.add(target)


# Set the desired end-effector pose to the location of target
Tep_1 = panda.fkine(panda.q)
Tep_1.A[:3, 3] = target_1.T[:3, 3] # get only position x y z
Tep_2 = panda.fkine(panda.q)
Tep_2.A[:3, 3] = target_2.T[:3, 3] # get only position x y z
#Tep.A[2, 3] += 0.1

#Tep.A[2, 3] += 0.1

#Show axis
# end-effector axes
ee_axes = sg.Axes(0.1)
# This pose can be either a spatialmat SE3 or a 4x4 ndarray
ee_axes.T = panda.fkine(panda.q)
# goal axes
goal_axes_1 = sg.Axes(0.1)
goal_axes_1.T = Tep_1
# goal axes 1=2
goal_axes_2 = sg.Axes(0.1)
goal_axes_2.T = Tep_2
# Add the axes to the environment
env.add(ee_axes)
env.add(goal_axes_1) 
env.add(goal_axes_2) 

10

In [68]:
panda

ERobot: panda (by Franka Emika), 7 joints (RRRRRRR), 1 gripper, geometry, collision
┌─────┬──────────────┬───────┬─────────────┬────────────────────────────────────────────────┐
│link │     link     │ joint │   parent    │              ETS: parent to link               │
├─────┼──────────────┼───────┼─────────────┼────────────────────────────────────────────────┤
│   0 │ panda_link0  │       │ BASE        │                                                │
│   1 │ panda_link1  │     0 │ panda_link0 │ SE3(0, 0, 0.333) ⊕ Rz(q0)                      │
│   2 │ panda_link2  │     1 │ panda_link1 │ SE3(-90°, -0°, 0°) ⊕ Rz(q1)                    │
│   3 │ panda_link3  │     2 │ panda_link2 │ SE3(0, -0.316, 0; 90°, -0°, 0°) ⊕ Rz(q2)       │
│   4 │ panda_link4  │     3 │ panda_link3 │ SE3(0.0825, 0, 0; 90°, -0°, 0°) ⊕ Rz(q3)       │
│   5 │ panda_link5  │     4 │ panda_link4 │ SE3(-0.0825, 0.384, 0; -90°, -0°, 0°) ⊕ Rz(q4) │
│   6 │ panda_link6  │     5 │ panda_link5 │ SE3(90°, -0°, 0°) ⊕ Rz(q5

In [4]:
panda.link_dict["panda_link1"]

Link("panda_link1", SE3(0, 0, 0.333) ⊕ Rz(q), parent="panda_link0", qlim=[-2.9, 2.9])

In [5]:
panda.link_dict["panda_hand"]

Link("panda_hand", SE3(0°, -0°, -45°), parent="panda_link8")

In [6]:
def move(Tep):
    arrived = False
    while not arrived:
        # The pose of the Panda's end-effector
        Te = panda.fkine(panda.q)

        # Transform from the end-effector to desired pose
        eTep = Te.inv() * Tep

        # Spatial error
        e = np.sum(np.abs(np.r_[eTep.t, eTep.rpy() * np.pi / 180]))

        # Calulate the required end-effector spatial velocity for the robot
        # to approach the goal. Gain is set to 1.0
        v, arrived = rtb.p_servo(Te, Tep, 1.0, 0.01)

        # Gain term (lambda) for control minimisation
        Y = 0.01

        # Quadratic component of objective function
        Q = np.eye(n + 6)

        # Joint velocity component of Q
        Q[:n, :n] *= Y

        # Slack component of Q
        Q[n:, n:] = (1 / e) * np.eye(6)

        # The equality contraints
        Aeq = np.c_[panda.jacobe(panda.q), np.eye(6)]
        beq = v.reshape((6,))

        # The inequality constraints for joint limit avoidance
        Ain = np.zeros((n + 6, n + 6))
        bin = np.zeros(n + 6)

        # The minimum angle (in radians) in which the joint is allowed to approach
        # to its limit
        ps = 0.05

        # The influence angle (in radians) in which the velocity damper
        # becomes active
        pi = 0.9

        # Form the joint limit velocity damper
        Ain[:n, :n], bin[:n] = panda.joint_velocity_damper(ps, pi, n)

        # For each collision in the scene
        for collision in collisions:

            # Form the velocity damper inequality contraint for each collision
            # object on the robot to the collision in the scene
            c_Ain, c_bin = panda.link_collision_damper(
                collision,
                panda.q[:n],
                0.3,
                0.05,
                1.0,
                start=panda.link_dict["panda_link1"],
                end=panda.link_dict["panda_hand"],
            )

            # If there are any parts of the robot within the influence distance
            # to the collision in the scene
            if c_Ain is not None and c_bin is not None:
                c_Ain = np.c_[c_Ain, np.zeros((c_Ain.shape[0], 6))]

                # Stack the inequality constraints
                Ain = np.r_[Ain, c_Ain]
                bin = np.r_[bin, c_bin]

        # Linear component of objective function: the manipulability Jacobian
        c = np.r_[-panda.jacobm(panda.q).reshape((n,)), np.zeros(6)]

        # The lower and upper bounds on the joint velocity and slack variable
        lb = -np.r_[panda.qdlim[:n], 10 * np.ones(6)]
        ub = np.r_[panda.qdlim[:n], 10 * np.ones(6)]

        # Solve for the joint velocities dq
        qd = qp.solve_qp(Q, c, Ain, bin, Aeq, beq, lb=lb, ub=ub, solver='osqp')

        # Apply the joint velocities to the Panda
        panda.qd[:n] = qd[:n]

        # Step the simulator by 50 ms
        env.step(0.01)


In [7]:
while(1):
    move(Tep_1)
    move(Tep_2)

pybullet build time: May 20 2022 19:44:17
/home/robotlab/anaconda3/envs/rbt/lib/python3.8/site-packages/qpsolvers/conversions/ensure_sparse_matrices.py:38: UserWarning: Converted P to scipy.sparse.csc.csc_matrix
For best performance, build P as a scipy.sparse.csc_matrix rather than as a numpy.ndarray
  warnings.warn(
/home/robotlab/anaconda3/envs/rbt/lib/python3.8/site-packages/qpsolvers/conversions/ensure_sparse_matrices.py:38: UserWarning: Converted G to scipy.sparse.csc.csc_matrix
For best performance, build G as a scipy.sparse.csc_matrix rather than as a numpy.ndarray
  warnings.warn(
/home/robotlab/anaconda3/envs/rbt/lib/python3.8/site-packages/qpsolvers/conversions/ensure_sparse_matrices.py:38: UserWarning: Converted A to scipy.sparse.csc.csc_matrix
For best performance, build A as a scipy.sparse.csc_matrix rather than as a numpy.ndarray
  warnings.warn(
/home/robotlab/anaconda3/envs/rbt/lib/python3.8/site-packages/qpsolvers/solvers/osqp_.py:161: UserWarning: OSQP exited with sta

TypeError: 'NoneType' object is not subscriptable